<a href="https://colab.research.google.com/github/jtuckayo/scratchpad/blob/main/DuckDB_%26_Iceberg_The_Future_of_Lightweight_Data_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a recreation of the notebook used during the [live discussion](https://www.youtube.com/live/rjtgxz3A0f8?si=54XYo1qT-vfkyeU7) between Brian Olsen from [Tabular](https://https://www.tabular.io/) and Mehdi Ouazza from [Mother Duck](https://https://motherduck.com/) on [Apache Iceberg](https://iceberg.apache.org/) and how to use it with [DuckDB](https://duckdb.org/).

You can use it while watching the [video](https://www.youtube.com/live/rjtgxz3A0f8?si=54XYo1qT-vfkyeU7).



In [ ]:
# prompt: install duckdb and sqlite in this notebook

!pip install duckdb
!pip install pysqlite3


In [ ]:
import duckdb

In [ ]:
pytable = duckdb.sql("SELECT * FROM 's3://us-prd-motherduck-open-datasets/who_ambient_air_quality/csv/who_ambient_air_quality_database_version_v6_april_2023.csv'").arrow()

In [ ]:
pytable

In [ ]:
# prompt: install pyiceberg

!pip install pyiceberg


In [ ]:
!mkdir "/content/warehouse"

In [ ]:
from pyiceberg.catalog.sql import SqlCatalog

warehouse_path = "/content/warehouse"
catalog = SqlCatalog(
    "default",
    **{
      "uri":f"sqlite:///{warehouse_path}/pyiceberg_catalog_db",
      "warehouse":f"file://{warehouse_path}",
    },
)

In [ ]:
catalog.create_namespace("default")

table = catalog.create_table(
    "default.air_quality",
    schema = pytable.schema,
)

In [ ]:
table.append(pytable)
len(table.scan().to_arrow())

In [ ]:
con_ice = table.scan().to_duckdb(table_name = "air")

In [ ]:
con_ice.sql('FROM air;')

In [ ]:
df = table.scan(row_filter="city == 'Berlin'").to_arrow()

In [ ]:
table.overwrite(df)

In [ ]:
con_ice = table.scan().to_duckdb(table_name = "air")

In [ ]:
con_ice.sql('FROM air;')